<a href="https://colab.research.google.com/github/Vengalagagan/NLP/blob/main/2403a52222_NLP(Assignment_6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load text data

In [2]:
import pandas as pd
data=pd.read_excel('/content/LDA-Data.xlsx')

data.head()


,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


Text Preprocesing: Clean Text, Word Tokenization, stopword removal, lemma,Rejoin

In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
def nltk_preprocessing(text):
    # Initialize NLTK tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))


    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    text = text.lower()  # Convert to lowercase
    tokenized_words = word_tokenize(text) # Corrected indentation

    # 3. Stopword Removal
    filtered_words = [word for word in tokenized_words if word not in stop_words]

    # 4. Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # 5. Rejoin words
    clean_txt= ' '.join(lemmatized_words)

    return clean_txt

print("NLTK preprocessing  function created successfully!")

NLTK preprocessing  function created successfully!


In [5]:
# Example usage of the nltk_preprocessing function
sample_text = data['News'].iloc[0] # Select the first news article from the Series
processed_text = nltk_preprocessing(sample_text)
print(f"Original text: {sample_text}")
print(f"Processed text: {processed_text}")

Original text: Virat scored century in match
Processed text: virat scored century match


In [6]:
data['clean_News'] = data['News'].apply(nltk_preprocessing)
print("\nComparison of previous clean_txt and new clean_txt_pipeline (first 5 rows):")
print(data[['News', 'clean_News']].head())



Comparison of previous clean_txt and new clean_txt_pipeline (first 5 rows):
                             News                      clean_News
0   Virat scored century in match      virat scored century match
1            BJP won in elections                    bjp election
2  Bumra took 5 wicket in a match       bumra took 5 wicket match
3  Congress form state government  congress form state government


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words='english',
    max_df=0.95,
    min_df=1
)

doc_term_matrix = vectorizer.fit_transform(data['clean_News'])


In [8]:
print(pd.DataFrame(doc_term_matrix.toarray(),
                   columns=vectorizer.get_feature_names_out()).head())

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [9]:
def display_topics(model, words, n):
    for i in range(len(model.components_)):
        print("Topic", i)
        for j in model.components_[i].argsort()[-n:]:
            print(words[j], end=" ")
        print("\n")


Bow

In [14]:
document_topics = lda_model.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics:")
print(data[['clean_News', 'topic']].head())


DataFrame with assigned topics:
                       clean_News  topic
0      virat scored century match      1
1                    bjp election      0
2       bumra took 5 wicket match      1
3  congress form state government      2


In [17]:
fn = vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(), columns=fn)
print(bow_df)

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


Apply Lda

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=3, random_state=42)

lda_model.fit(doc_term_matrix)
print("LDA model fitted successfully with 3 components.")

LDA model fitted successfully with 3 components.


In [12]:
feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names, 3)

Topic 0
bumra bjp election 

Topic 1
virat scored match 

Topic 2
congress form state 



Identify words for  each topic


In [20]:
print("\n Identified words for each topic:")
feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names, 3)


 Identified words for each topic:
Topic 0
bumra bjp election 

Topic 1
virat scored match 

Topic 2
congress form state 



Topic modeling

In [21]:
print("\n Topic modeling (document topic assignments):")
print(data[['clean_News', 'topic']].head())


 Topic modeling (document topic assignments):
                       clean_News  topic
0      virat scored century match      1
1                    bjp election      0
2       bumra took 5 wicket match      1
3  congress form state government      2
